# Importando datos de twitter con python y tweepy

<img src="./images/tweepy.png" alt="Twitter logo" style="width: 800px;" align="left"/>

[Twitter](https://twitter.com) proporciona un API que permite descargar datos de su red social. Para ello vamos a utilizar python y la libreria [tweepy](https://github.com/tweepy/tweepy).

El objetivo es recuperar tweets asociados a la palabra 'NoSQL' y almacenarlos en un fichero para su posterior análisis.

Lo primero que hay que hacer para acceder al API de Twitter es [registrar](https://www.google.es/webhp?q=crear+twitter+app) una nueva aplicación a través de la página [Twitter Application Management](https://apps.twitter.com/) identificandonos con nuestro usuario y contraseña.

Al registrar aplicación, Twitter nos proporciona las claves que nos hacen falta para acceder ...

In [1]:
consumer_key = 'hRNtRgjzGq3wq3mt3fbuUkQ2c'
consumer_secret = 'yBbXnvNRpm92wvblpG9xhUMFF7w9sgxLfQT8k15Fs3k1RN4pnQ'
access_token_key = '12391902-qHO3gUBIvKuv7DjajXBmdm2SyZH8vgmR3jcpLVnnM'
access_token_secret = '9ViwfNW5FhOLhahaf4qimDLXfYuqDtGzJ1MmAQM0gN3LK'

El siguiente paso es importar la libreria de tweepy y logarnos en twitter.

In [2]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)

api = tweepy.API(auth)

## Probamos la libreria 

Para probar la librería vamos a recuperar información sobre un usuario de twitter, su nombre, sus followers, etc..

In [3]:
user = api.get_user('NoSQLDigest')

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:100: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [4]:
print "Nombre:", user.screen_name
print "Descripción:", user.description
print "Nº de followers:", user.followers_count
print "Nº de amigos:", user.friends_count
print "Nº de tweets: ", user.statuses_count


Nombre: NoSQLDigest
Descripción: NoSQL Digest of tweets.
Nº de followers: 9791
Nº de amigos: 12
Nº de tweets:  668135


## Recuperando Tweets mediante un término de búsqueda

In [6]:
lookup ='NoSQL'

Mediante el siguiente método podemos recuperar rápidamente tweets,  aunque dependemos del límite que Twitter ha establecido 

In [7]:
max_tweets = 100
search_results = api.search(q=lookup, lang = 'es', count=max_tweets)

print len(search_results)


60


/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:100: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [8]:
from prettytable import PrettyTable

table = PrettyTable(["User", "Fecha", "Texto"])
table.align["User"] = "l"
table.align["Texto"] = "l"

for tweet in search_results[0:10]:
    table.add_row([tweet.user.screen_name, tweet.created_at, tweet.text[:80]])
 
print table

+----------------+---------------------+----------------------------------------------------------------------------------+
| User           |        Fecha        | Texto                                                                            |
+----------------+---------------------+----------------------------------------------------------------------------------+
| shermanMG      | 2015-10-20 04:15:29 | 11:14 y encontré un vídeo de NoSQL. Que verga, sueño.                            |
| _Juankar_      | 2015-10-18 19:21:15 | #MundoSQL Tipos de datos en #MongoDB http://t.co/IyQv4N4HLf #mongodb #nosql      |
| cuervocr       | 2015-10-18 18:13:08 | ¿Qué tendencias para desarrolladores habrá tener en cuenta en 2016?: De contened |
| _Juankar_      | 2015-10-18 13:21:04 | #MundoSQL Cambiando la estructura de una coleccion de #MongoDB http://t.co/BO2yW |
| _Juankar_      | 2015-10-18 10:16:53 | #MundoSQL Seleccionar los campos a visualizar en las consultas en #MongoDB http: |
| tmj_sf

## Recuperando el time line de un usuario


In [9]:
timeline_results = api.user_timeline(screen_name = 'NoSQLDigest', count = 1000, include_rts = True)
len(timeline_results)

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:100: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


198

## Recuperando tweets mediante un cursor

Este método utiliza un curso que posibilita saltarnos la restricción de 100 tweets ...

In [10]:
c = tweepy.Cursor(api.search, q= lookup).items()
    
search_results = []
while True:
    try:
        tweet = c.next()
        # Insert into db
        search_results.append(tweet)
    except tweepy.TweepError as e:
        print e
        break

print len(search_results)

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:100: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:100: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:100: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause ce

{"errors":[{"message":"Rate limit exceeded","code":88}]}
2645


Aunque vemos que tenemos otra. En este caso es por tiempo :-(

## Recuperando el time line mediante un cursor

In [11]:
import sys

c = tweepy.Cursor(api.user_timeline,id='NoSQLDigest').items()    
timeline_results = []
while True:
    try:
        tweet = c.next()
        # Insert into db
        timeline_results.append(tweet)
    except:
        print "Error: ", sys.exc_info()[0]
        break

print len(timeline_results)

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:100: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:100: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:100: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause ce

Error:  <type 'exceptions.StopIteration'>
3138


## Visualizando el contenido de un Tweet

Twitter devuelve datos en formato [JSON](http://www.json.org/json-es.html). Vamos a ver que pinta tiene 1 tweet:

In [12]:
import pprintpp

tweet = search_results[0]
pprintpp.pprint(tweet._json)

{
    u'contributors': None,
    u'coordinates': None,
    u'created_at': u'Tue Oct 20 05:19:55 +0000 2015',
    u'entities': {
        u'hashtags': [{u'indices': [136, 140], u'text': u'Book'}],
        u'symbols': [],
        u'urls': [
            {
                u'display_url': u'is.gd/0OAJej',
                u'expanded_url': u'http://is.gd/0OAJej',
                u'indices': [112, 135],
                u'url': u'https://t.co/evdHW8DWUn',
            },
        ],
        u'user_mentions': [
            {
                u'id': 341552391,
                u'id_str': u'341552391',
                u'indices': [3, 11],
                u'name': u'Azat Mardan',
                u'screen_name': u'azat_co',
            },
        ],
    },
    u'favorite_count': 0,
    u'favorited': False,
    u'geo': None,
    u'id': 656339038765379584,
    u'id_str': u'656339038765379584',
    u'in_reply_to_screen_name': None,
    u'in_reply_to_status_id': None,
    u'in_reply_to_status_id_str': None,


## Parseando el resultado

El objetivo de las siguientes funciones es simplificar la información que vamoso a guardar en fichero, ya que Twitter suministrar mucha información:

In [13]:
def parse_user(usr):
    user = {}  
    user["created_at"] = usr['created_at']
    user["description"] = usr['description']
    user["favourites_count"] = usr['favourites_count']
    user["followers_count"] = usr['followers_count']
    user["friends_count"] = usr['friends_count']
    user["geo_enabled"] = usr['geo_enabled']
    user["_id"] = usr['id']
    user["id_str"] =usr['id_str']
    user["name"] = usr['name']
    user["screen_name"] = usr['screen_name']
    user["statuses_count"] = usr['statuses_count']
    user["time_zone"] = usr['time_zone']
    return user

In [14]:
def parse_tweet(t):
    tweet = {}
    tweet['created_at'] = t['created_at']
    #for ht in tweet.entities.hashtags:
    #    print ht.text

    tweet['entities'] = []
    for k in t['entities']['hashtags']:
        tweet['entities'].append(k['text'])
  
    tweet['user_mentions'] = []
    for k in t['entities']['user_mentions']:
        k.pop("indices", None)
        tweet['user_mentions'].append(k)

    tweet['favorite_count'] =  t['favorite_count']

    if t['geo'] <> None:
        tweet['geo'] = t['geo']

    tweet['_id'] = t['id']
    tweet['id_str'] = t['id_str']  

    tweet['lang'] = t['lang']
    tweet['retweet_count'] = t['retweet_count']
    tweet['source'] = t['source']
    tweet['text'] = t['text']
    tweet['user'] = parse_user(t['user'])

    if 'retweeted_status' in t.keys():
        rs = t['retweeted_status']

        retweeted_status = {}
        retweeted_status['created_at'] = rs['created_at']

        if rs['geo'] <> None:
            retweeted_status['geo'] = rs['geo']

        retweeted_status['id'] = rs['id']
        retweeted_status['id_str'] = rs['id_str']  

        retweeted_status['user'] = parse_user(rs['user'])
        tweet['retweeted_status'] = retweeted_status
        
    return tweet

Parseamos el contenido que hemos descargado previamente ...

In [15]:
tweets = []

for tweet in search_results:
    tweets.append(parse_tweet(tweet._json))
    

Y visualiamos el contenido. Tambien en formato JSON pero mucho más simple

In [16]:
pprintpp.pprint(tweets[0])

{
    '_id': 656339038765379584,
    'created_at': u'Tue Oct 20 05:19:55 +0000 2015',
    'entities': [u'Book'],
    'favorite_count': 0,
    'id_str': u'656339038765379584',
    'lang': u'en',
    'retweet_count': 1,
    'retweeted_status': {
        'created_at': u'Tue Oct 20 02:17:39 +0000 2015',
        'id': 656293169206726656,
        'id_str': u'656293169206726656',
        'user': {
            '_id': 341552391,
            'created_at': u'Sun Jul 24 15:15:36 +0000 2011',
            'description': u'Technology fellow at @CapitalOne, published author of several books and JavaScript/Node.js expert',
            'favourites_count': 1357,
            'followers_count': 2538,
            'friends_count': 201,
            'geo_enabled': True,
            'id_str': u'341552391',
            'name': u'Azat Mardan',
            'screen_name': u'azat_co',
            'statuses_count': 18533,
            'time_zone': u'Pacific Time (US & Canada)',
        },
    },
    'source': u'<a hre

## Grabando la información en un fichero

Por último vamos a grabar la información descargada en un fichero, de forma que la podamos analizar más tarde

In [17]:
import json
    
with open('./data/tweets.json',"w") as file:
    for t in tweets:
        r = json.dumps(t)
        file.write(r)
        file.write("\n")

In [18]:
print len(tweets)

2645


In [19]:
tweets = []
for tweet in timeline_results:
    tweets.append(parse_tweet(tweet._json))
    
with open('./data/timeline.json',"w") as file:
    for t in tweets:
        r = json.dumps(t)
        file.write(r)
        file.write("\n")
        
print len(tweets)

3138
